# Install Taipy

To install Taipy, just `pip install` it.

In [1]:
%pip install taipy -q

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install pmdarima -q

Note: you may need to restart the kernel to use updated packages.


# Import the packages

In [3]:
from taipy.gui import Gui, Markdown, notify
from taipy import Config, Scope
import taipy as tp

import datetime as dt

from pmdarima import auto_arima

from sklearn.linear_model import LinearRegression

import pandas as pd
import numpy as np

# Taipy Gui Basics
## Markdown Syntax

Taipy uses the Markdown syntax to display elements. `#` creates a title, `*` puts your text in italics and `**` puts it in bold.

![](img/gui_basic_eng.png)

In [4]:
page_md = """
# Taipy

Test **here** to put some *markdown*

Click to access the [doc](https://docs.taipy.io/en/latest/)
"""

In [5]:
Gui(page_md).run(dark_mode=False, run_browser=False)

 * Server starting on http://127.0.0.1:5000


## Visual elements
Create different visual elements. The syntax is always the same for each visual element.  `<|{value}|name_of_visual_element|property_1=value_of_property_1|...|>`
- Create a [slider](https://docs.taipy.io/en/latest/manuals/gui/viselements/slider/) `<|{value}|slider|>`

- Create a [date](https://docs.taipy.io/en/latest/manuals/gui/viselements/date/) `<|{value}|date|>`

- Create a [selector](https://docs.taipy.io/en/latest/manuals/gui/viselements/selector/) `<|{value}|selector|lov={list_of_values}|>`


![](img/control.png)

In [6]:
slider_value = 0
date_value = None
selected_value = None
selector = ['Test 1', 'Test 2', 'Test 3']

control_md = """
## Controls

<|{slider_value}|slider|> <|{slider_value}|>

<|{date_value}|date|> <|{date_value}|>

<|{selected_value}|selector|lov={selector}|> <|{selected_value}|>
"""

In [7]:
Gui(control_md).run(dark_mode=False, run_browser=False)

Gui server has been stopped.
 * Server starting on http://127.0.0.1:5000


## Data Viz

A dataset gathering information on the number of deaths, confirmed cases and recovered for different regions is going to be used to create an interactive Dashboard.

In [8]:
path_to_data = "data/covid-19-all.csv"
data = pd.read_csv(path_to_data, low_memory=False)
data[-5:]

,Country/Region,Province/State,Latitude,Longitude,Confirmed,Recovered,Deaths,Date
1241947,Vietnam,NaN,14.058324,108.277199,1465.0,1325.0,35.0,2020-12-31
1241948,West Bank and Gaza,NaN,31.952200,35.233200,138004.0,117183.0,1400.0,2020-12-31
1241949,Yemen,NaN,15.552727,48.516388,2099.0,1394.0,610.0,2020-12-31
1241950,Zambia,NaN,-13.133897,27.849332,20725.0,18660.0,388.0,2020-12-31
1241951,Zimbabwe,NaN,-19.015438,29.154857,13867.0,11250.0,363.0,2020-12-31


In [9]:
def initialize_case_evolution(data, selected_country='France') -> pd.DataFrame:
    # Aggregation of the dataframe per Country/Region
    country_date_df = data.groupby(["Country/Region",'Date']).sum().reset_index()
    
    # a country is selected, here France by default
    country_date_df = country_date_df.loc[country_date_df['Country/Region']==selected_country]
    return country_date_df

In [10]:
country_date_df = initialize_case_evolution(data)
country_date_df.head()

C:\Users\jacta\AppData\Local\Temp\ipykernel_41444\445871222.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  country_date_df = data.groupby(["Country/Region",'Date']).sum().reset_index()


,Country/Region,Date,Latitude,Longitude,Confirmed,Recovered,Deaths
18101,France,2020-01-24,14.6415,-56.3159,2.0,0.0,0.0
18102,France,2020-01-25,14.6415,-56.3159,3.0,0.0,0.0
18103,France,2020-01-26,14.6415,-56.3159,3.0,0.0,0.0
18104,France,2020-01-27,14.6415,-56.3159,3.0,0.0,0.0
18105,France,2020-01-28,14.6415,-56.3159,4.0,0.0,0.0


Create a [chart](https://docs.taipy.io/en/latest/manuals/gui/viselements/chart/) showing the evolution of Deaths in France (_Deaths_ for _y_ and _Date_ for _x_). The visual element (chart) has the same syntax as the other ones with specific properties (_x_, _y_, _type_ for example). Here are some [examples of charts](https://docs.taipy.io/en/release-1.1/manuals/gui/viselements/charts/bar/). The _x_ and _y_ porperties only needs the name of the dataframe columns to display.

![](img/simple_graph.png)

In [11]:
country_md = "<|{country_date_df}|chart|x=Date|y=Deaths|type=bar|>"

In [12]:
Gui(country_md).run(dark_mode=False, run_browser=False)

Gui server has been stopped.
 * Server starting on http://127.0.0.1:5000


## Add new traces

- Add on the graph the number of Confirmed and Recovered cases (_Confirmed_ and _Recovered_) with the number of Deaths
- _y_ (and _x_) can be indexed this way to add more traces (`y[1]=`, `y[2]=`, `y[3]=`).

![](img/multi_traces.png)

In [13]:
country_md = "<|{country_date_df}|chart|type=bar|x=Date|y[1]=Deaths|y[2]=Recovered|y[3]=Confirmed|>"

In [14]:
Gui(country_md).run(dark_mode=False, run_browser=False)

Gui server has been stopped.
 * Server starting on http://127.0.0.1:5000


## Style the graph with personalized properties
The _layout_ dictionnary specifies how bars should be displayed. They would be 'stacked'.

The _options_ dictionary will change the opacity of the unselected markers.

These are Plotly properties.

![](img/stack_chart.png)

In [15]:
layout = {'barmode':'stack'}
options = {"unselected":{"marker":{"opacity":0.5}}}
country_md = "<|{country_date_df}|chart|type=bar|x=Date|y[1]=Deaths|y[2]=Recovered|y[3]=Confirmed|layout={layout}|options={options}|>"

In [16]:
Gui(country_md).run(dark_mode=False, run_browser=False)

Gui server has been stopped.
 * Server starting on http://127.0.0.1:5000


## Add texts that sums up the data

Use the [text](https://docs.taipy.io/en/latest/manuals/gui/viselements/text/) visual element.

- Add the total number of Deaths (last line of _country_date_df_)
- Add the total number of Recovered (last line of _country_date_df_)
- Add the total number of Confirmed (last line of _country_date_df_)


In [17]:
country_date_df

,Country/Region,Date,Latitude,Longitude,Confirmed,Recovered,Deaths
18101,France,2020-01-24,14.641500,-56.315900,2.0,0.0,0.0
18102,France,2020-01-25,14.641500,-56.315900,3.0,0.0,0.0
18103,France,2020-01-26,14.641500,-56.315900,3.0,0.0,0.0
18104,France,2020-01-27,14.641500,-56.315900,3.0,0.0,0.0
18105,France,2020-01-28,14.641500,-56.315900,4.0,0.0,0.0
...,...,...,...,...,...,...,...
18439,France,2020-12-27,91.463495,-238.646914,2616510.0,195861.0,62867.0
18440,France,2020-12-28,91.463495,-238.646914,2619616.0,196642.0,63235.0
18441,France,2020-12-29,91.463495,-238.646914,2631110.0,197726.0,64204.0
18442,France,2020-12-30,91.463495,-238.646914,2657624.0,198966.0,64508.0


This is how we can get the total number of Deaths from the dataset for France.

In [18]:
country_date_df.iloc[-1, 6] # gives the number of deaths for France (5 is for recovered and 4 is confirmed)

64759.0

Use the [text](https://docs.taipy.io/en/release-1.1/manuals/gui/viselements/text/) visual element. Note that between `{}`, any Python variable can be put but also any Python code.

![](img/control_text.png)

In [19]:
country_md = """
## Deaths <|{country_date_df.iloc[-1, 6]}|>

## Recovered <|{country_date_df.iloc[-1, 5]}|text|>

## Confirmed <|{country_date_df.iloc[-1, 4]}|text|>

<|{country_date_df}|chart|type=bar|x=Date|y[1]=Deaths|y[2]=Recovered|y[3]=Confirmed|layout={layout}|options={options}|>
"""

In [20]:
Gui(country_md).run(dark_mode=False, run_browser=False)

Gui server has been stopped.
 * Server starting on http://127.0.0.1:5000


## Local _on_change_

- Add a [selector](https://docs.taipy.io/en/latest/manuals/gui/viselements/selector/) with `dropdown=True` containing the name of all the _Country/region_
- Give to the _on_change_ selector property the name of the _on_change_country_ function. This function will be called when the selector will be used.
- This function has a 'state' parameter and has to be completed. When the selector is used, this function is called with the _state_ argument. It contains all the Gui variables; 'state.country_date_df' is then the dataframe used in the Gui.

![](img/on_change_local.png)

In [21]:
country_lov = sorted(data["Country/Region"].dropna().unique().tolist())
selected_country = "France"

final_country_md = """
<|layout|columns=1 1 1 1|
<|{selected_country}|selector|lov={country_lov}|on_change=on_change_country|dropdown|label=Country|>

## Deaths <|{country_date_df.iloc[-1, 6]}|>

## Recovered <|{country_date_df.iloc[-1, 5]}|>

## Confirmed <|{country_date_df.iloc[-1, 4]}|>
|>

<|{country_date_df}|chart|type=bar|x=Date|y[1]=Deaths|y[2]=Recovered|y[3]=Confirmed|layout={layout}|options={options}|>
"""

In [22]:
def on_change_country(state):
    # state contains all the Gui variables and this is through this state variable that we can update the Gui
    # state.selected_country, state.country_date_df, ...
    # update country_date_df with the right country (use initialize_case_evolution)
    print("Chosen country: ", state.selected_country)
    state.country_date_df = initialize_case_evolution(data, state.selected_country)

In [23]:
Gui(final_country_md).run(dark_mode=False, run_browser=False)

Gui server has been stopped.
 * Server starting on http://127.0.0.1:5000


# Map

In [24]:
def initialize_map(data):
    data['Province/State'] = data['Province/State'].fillna(data["Country/Region"])
    data_province = data.groupby(["Country/Region",
                                  'Province/State',
                                  'Longitude',
                                  'Latitude'])\
                         .max()

    data_province_displayed = data_province[data_province['Deaths']>10].reset_index()

    data_province_displayed['Size'] = np.sqrt(data_province_displayed.loc[:,'Deaths']/data_province_displayed.loc[:,'Deaths'].max())*80 + 3
    data_province_displayed['Text'] = data_province_displayed.loc[:,'Deaths'].astype(str) + ' deaths</br>' + data_province_displayed.loc[:,'Province/State']
    return data_province_displayed

In [25]:
data_province_displayed = initialize_map(data)
data_province_displayed.head()

,Country/Region,Province/State,Longitude,Latitude,Confirmed,Recovered,Deaths,Date,Size,Text
0,Afghanistan,Afghanistan,67.709953,33.93911,51526.0,41727.0,2191.0,2020-12-31,17.771247,2191.0 deaths</br>Afghanistan
1,Albania,Albania,20.168300,41.15330,58316.0,33634.0,1181.0,2020-12-31,13.844784,1181.0 deaths</br>Albania
2,Algeria,Algeria,1.659600,28.03390,99610.0,67127.0,2756.0,2020-12-31,19.566684,2756.0 deaths</br>Algeria
3,Andorra,Andorra,1.521800,42.50630,8049.0,7432.0,84.0,2020-12-31,5.892249,84.0 deaths</br>Andorra
4,Angola,Angola,17.873900,-11.20270,17553.0,11044.0,405.0,2020-12-31,9.350728,405.0 deaths</br>Angola


Properties to style the map
- marker color corresponds to the number of Deaths (column _Deaths_)
- marker sizes corresponds to the size in _Size_ column which is created from the number of Deaths

layout_map permet defined the initial zoom and position of the map


In [26]:
marker_map = {"color":"Deaths", "size": "Size", "showscale":True, "colorscale":"Viridis"}
layout_map = {
            "dragmode": "zoom",
            "mapbox": { "style": "open-street-map", "center": { "lat": 38, "lon": -90 }, "zoom": 3}
            }

We give to Plotly:
- a map type
- the name of the latitude column
- the name of the longitude column
- properties: on the size and color of the markers
- the name of the column for the text of the points

In [27]:
selected_points = []
map_md = """
<|{data_province_displayed}|chart|type=scattermapbox|selected={selected_points}|lat=Latitude|lon=Longitude|marker={marker_map}|layout={layout_map}|text=Text|mode=markers|height=800px|options={options}|>
"""

In [28]:
Gui(map_md).run(dark_mode=False, run_browser=False)

Gui server has been stopped.
 * Server starting on http://127.0.0.1:5000


![](img/carte.png)

# Taipy Core
Here are the functions that we are going to use to predict the number of Deaths for a country.
We will:
- preprocess the data (_preprocess_),
- create a training and testing database (_make_train_test_data_),
- train a model (_train_model_),
- generate predictions (_forecast_),
- generate a dataframe with the historical data and the predictions (_result_)

![](img/all_architecture.svg)

In [29]:
# initialise variables
selected_scenario = None
scenario_selector = None

first_date = dt.datetime(2020,11,1)

scenario_name = None

result = None

In [30]:
# This simple line will configure the execution of the code in parallel
#Config.configure_job_executions(mode="standalone", nb_of_workers=2)

In [31]:
def add_features(data):
    dates = pd.to_datetime(data["Date"])
    data["Months"] = dates.dt.month
    data["Days"] = dates.dt.isocalendar().day
    data["Week"] = dates.dt.isocalendar().week
    data["Day of week"] = dates.dt.dayofweek
    return data

def create_train_data(final_data, date):
    bool_index = pd.to_datetime(final_data['Date']) <= date
    train_data = final_data[bool_index]
    return train_data

def preprocess(initial_data, country, date):
    data = initial_data.groupby(["Country/Region",'Date'])\
                       .sum()\
                       .dropna()\
                       .reset_index()

    final_data = data.loc[data['Country/Region']==country].reset_index(drop=True)
    final_data = final_data[['Date','Deaths']]
    final_data = add_features(final_data)
    
    train_data = create_train_data(final_data, date)
    return final_data, train_data


def train_arima(train_data):
    model = auto_arima(train_data['Deaths'],
                       start_p=1, start_q=1,
                       max_p=5, max_q=5,
                       start_P=0, seasonal=False,
                       d=1, D=1, trace=True,
                       error_action='ignore',  
                       suppress_warnings=True)
    model.fit(train_data['Deaths'])
    return model


def forecast(model):
    predictions = model.predict(n_periods=60)
    return np.array(predictions)


def result(final_data, predictions, date):
    dates = pd.to_datetime([date + dt.timedelta(days=i)
                            for i in range(len(predictions))])
    final_data['Date'] = pd.to_datetime(final_data['Date'])
    predictions = pd.concat([pd.Series(dates, name="Date"),
                             pd.Series(predictions, name="Predictions")], axis=1)
    return final_data.merge(predictions, on="Date", how="outer")

First we must define the Data Nodes then the tasks (associated to the Python function). Furthermore, we gather these tasks into different pipelines and these pipelines into a scenario.

A Data Node needs a **unique id**. If needed, the storage type can be changed for CSV and SQL. Other parameters are then needed.

### Data Nodes and Task for preprocess

<img src="img/preprocess.svg" alt="drawing" width="500"/>

In [32]:
initial_data_cfg = Config.configure_data_node(id="initial_data",
                                              storage_type="csv",
                                              path=path_to_data,
                                              cacheable=True,
                                              validity_period=dt.timedelta(days=5),
                                              scope=Scope.GLOBAL)

country_cfg = Config.configure_data_node(id="country", default_data="France",
                                         cacheable=True, validity_period=dt.timedelta(days=5))


date_cfg = Config.configure_data_node(id="date", default_data=dt.datetime(2020,10,10),
                                         cacheable=True, validity_period=dt.timedelta(days=5))

<img src="img/preprocess.svg" alt="drawing" width="500"/>

In [33]:
final_data_cfg =  Config.configure_data_node(id="final_data",
                                            cacheable=True, validity_period=dt.timedelta(days=5))


train_data_cfg =  Config.configure_data_node(id="train_data", cacheable=True, validity_period=dt.timedelta(days=5))


<img src="img/preprocess.svg" alt="drawing" width="500"/>

In [34]:
task_preprocess_cfg = Config.configure_task(id="task_preprocess_data",
                                           function=preprocess,
                                           input=[initial_data_cfg, country_cfg, date_cfg],
                                           output=[final_data_cfg,train_data_cfg])

### Rest of the graph

In [35]:
### Data Nodes and Task for train_arima

model_cfg = Config.configure_data_node(id="model", cacheable=True, validity_period=dt.timedelta(days=5), scope=Scope.PIPELINE)

task_train_cfg = Config.configure_task(id="task_train",
                                      function=train_arima,
                                      input=train_data_cfg,
                                      output=model_cfg)

### Data Nodes and Task for forecast

predictions_cfg = Config.configure_data_node(id="predictions", scope=Scope.PIPELINE)

task_forecast_cfg = Config.configure_task(id="task_forecast",
                                      function=forecast,
                                      input=model_cfg,
                                      output=predictions_cfg)

### Data Nodes and Task for result

result_cfg = Config.configure_data_node(id="result", scope=Scope.PIPELINE)

task_result_cfg = Config.configure_task(id="task_result",
                                      function=result,
                                      input=[final_data_cfg, predictions_cfg, date_cfg],
                                      output=result_cfg)

## [Configuration of pipelines](https://docs.taipy.io/en/release-1.1/manuals/reference/taipy.Config/#taipy.core.config.config.Config.configure_default_pipeline)

In [36]:
pipeline_preprocessing_cfg = Config.configure_pipeline(id="pipeline_preprocessing",
                                                       task_configs=[task_preprocess_cfg])

pipeline_arima_cfg = Config.configure_pipeline(id="ARIMA",
                                               task_configs=[task_train_cfg,
                                                             task_forecast_cfg,
                                                             task_result_cfg])

## Add more models

<img src="img/pipeline_linear_regression.svg" alt="drawing" width="500"/>

In [37]:
def train_linear_regression(train_data):    
    y = train_data['Deaths']
    X = train_data.drop(['Deaths','Date'], axis=1)
    
    model = LinearRegression()
    model.fit(X,y)
    return model

def forecast_linear_regression(model, date):
    dates = pd.to_datetime([date + dt.timedelta(days=i)
                            for i in range(60)])
    X = add_features(pd.DataFrame({"Date":dates}))
    X.drop('Date', axis=1, inplace=True)
    predictions = model.predict(X)
    return predictions


task_train_cfg = Config.configure_task(id="task_train",
                                      function=train_linear_regression,
                                      input=train_data_cfg,
                                      output=model_cfg)

task_forecast_cfg = Config.configure_task(id="task_forecast",
                                      function=forecast_linear_regression,
                                      input=[model_cfg, date_cfg],
                                      output=predictions_cfg)

pipeline_linear_regression_cfg = Config.configure_pipeline(id="LinearRegression",
                                               task_configs=[task_train_cfg,
                                                             task_forecast_cfg,
                                                             task_result_cfg])

## [Configuration of scénario](https://docs.taipy.io/en/release-1.1/manuals/reference/taipy.Config/#taipy.core.config.config.Config.configure_default_scenario)

In [38]:
scenario_cfg = Config.configure_scenario(id='scenario', pipeline_configs=[pipeline_preprocessing_cfg,
                                                                          pipeline_arima_cfg,
                                                                          pipeline_linear_regression_cfg])

## Creation and submit of scenario

In [39]:
scenario = tp.create_scenario(scenario_cfg, name='First Scenario')
tp.submit(scenario)

C:\Users\jacta\AppData\Local\R-MINI~1\envs\notebook\lib\site-packages\taipy\core\data\csv.py:161: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(self._path)


[2022-10-05 09:41:21,653][Taipy][INFO] job JOB_task_preprocess_data_0328f038-0e89-4b3b-a35f-f7468ba17316 is completed.


C:\Users\jacta\AppData\Local\Temp\ipykernel_41444\1815540409.py:16: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  .sum()\


[2022-10-05 09:41:21,957][Taipy][INFO] job JOB_task_train_89fd5a58-772a-463e-a644-7776dfb88ff9 is completed.
[2022-10-05 09:41:22,048][Taipy][INFO] job JOB_task_forecast_7d081b84-77d1-4904-9f7d-61da457d8912 is completed.
[2022-10-05 09:41:22,119][Taipy][INFO] job JOB_task_result_5db2c068-f01d-4639-bda2-9033d0e4c09a is completed.
[2022-10-05 09:41:22,302][Taipy][INFO] job JOB_task_train_dd55a92f-2530-4081-b0db-e560f19d0616 is completed.
[2022-10-05 09:41:22,384][Taipy][INFO] job JOB_task_forecast_946e707f-4aa8-4fd2-9d25-b3ec736b6a03 is completed.
[2022-10-05 09:41:22,440][Taipy][INFO] job JOB_task_result_9f91b7a1-38f6-4a8e-8ad6-cc76804ec148 is completed.


{'PIPELINE_pipeline_preprocessing_6b95373e-b472-4ebc-912b-d1ec90b3b8fc': [<taipy.core.job.job.Job at 0x257017adba0>],
 'PIPELINE_ARIMA_8a944c8c-a207-46eb-a52a-f696104281c1': [<taipy.core.job.job.Job at 0x25772232a10>,
 'PIPELINE_LinearRegression_a2695fc1-54b2-49f6-a43d-a8bf4081d910': [<taipy.core.job.job.Job at 0x2577216fd90>,
  <taipy.core.job.job.Job at 0x25704565870>]}

In [40]:
scenario.initial_data.read()

C:\Users\jacta\AppData\Local\R-MINI~1\envs\notebook\lib\site-packages\taipy\core\data\csv.py:161: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(self._path)


,Country/Region,Province/State,Latitude,Longitude,Confirmed,Recovered,Deaths,Date
0,NaN,NaN,NaN,NaN,51526.0,41727.0,2191.0,2021-01-01
1,NaN,NaN,NaN,NaN,58316.0,33634.0,1181.0,2021-01-01
2,NaN,NaN,NaN,NaN,99897.0,67395.0,2762.0,2021-01-01
3,NaN,NaN,NaN,NaN,8117.0,7463.0,84.0,2021-01-01
4,NaN,NaN,NaN,NaN,17568.0,11146.0,405.0,2021-01-01
...,...,...,...,...,...,...,...,...
1241947,Vietnam,NaN,14.058324,108.277199,1465.0,1325.0,35.0,2020-12-31
1241948,West Bank and Gaza,NaN,31.952200,35.233200,138004.0,117183.0,1400.0,2020-12-31
1241949,Yemen,NaN,15.552727,48.516388,2099.0,1394.0,610.0,2020-12-31
1241950,Zambia,NaN,-13.133897,27.849332,20725.0,18660.0,388.0,2020-12-31


In [41]:
scenario.train_data.read()

,Date,Deaths,Months,Days,Week,Day of week
0,2020-01-24,0.0,1,5,4,4
1,2020-01-25,0.0,1,6,4,5
2,2020-01-26,0.0,1,7,4,6
3,2020-01-27,0.0,1,1,5,0
4,2020-01-28,0.0,1,2,5,1
...,...,...,...,...,...,...
256,2020-10-06,32386.0,10,2,41,1
257,2020-10-07,32466.0,10,3,41,2
258,2020-10-08,32539.0,10,4,41,3
259,2020-10-09,32649.0,10,5,41,4


In [42]:
scenario.ARIMA.predictions.read()

array([40425.81351614, 40572.73161338, 40751.72048211, 40898.63857936,
       41045.5566766 , 41192.47477385, 41339.39287109, 41486.31096833,
       41633.22906558, 41812.21793431, 41959.13603155, 42106.0541288 ,
       42252.97222604, 42399.89032329, 42546.80842053, 42693.72651778,
       42872.71538651, 43019.63348375, 43166.551581  , 43313.46967824,
       43460.38777548, 43607.30587273, 43790.05127049, 43969.04013923,
       44115.95823647, 44262.87633371, 44409.79443096, 44556.7125282 ,
       44703.63062545, 44850.54872269, 45029.53759142, 45176.45568867,
       45323.37378591, 45470.29188316, 45617.2099804 , 45764.12807764,
       45911.04617489, 46090.03504362, 46236.95314086, 46383.87123811,
       46530.78933535, 46677.7074326 , 46824.62552984, 46971.54362709,
       47150.53249582, 47297.45059306, 47444.36869031, 47591.28678755,
       47738.20488479, 47885.12298204, 48032.04107928, 48211.02994801,
       48393.77534578, 48540.69344302, 48687.61154027, 48834.52963751,
      

## Caching
Some job are skipped because no change has been done to the "input" Data Nodes.

In [43]:
tp.submit(scenario)

[2022-10-05 09:41:23,248][Taipy][INFO] job JOB_task_preprocess_data_d8c81cbf-bcf9-47f2-890d-4d9c8c445a0b is skipped.
[2022-10-05 09:41:23,390][Taipy][INFO] job JOB_task_train_e58799aa-b5c3-4262-9320-2b2af529b2ff is skipped.
[2022-10-05 09:41:23,463][Taipy][INFO] job JOB_task_forecast_091c8640-9acc-4f2c-bf54-d0bd39d9446e is completed.
[2022-10-05 09:41:23,519][Taipy][INFO] job JOB_task_result_9e410cca-3706-4f17-8b69-c2e2e0586ead is completed.
[2022-10-05 09:41:23,657][Taipy][INFO] job JOB_task_train_ccb311e4-0184-496c-ade1-a82a62298504 is skipped.
[2022-10-05 09:41:23,730][Taipy][INFO] job JOB_task_forecast_f93d746e-618c-42c4-afa2-6636917dbedd is completed.
[2022-10-05 09:41:23,788][Taipy][INFO] job JOB_task_result_edaf3591-0296-4526-9dc4-15c7ebde5f0f is completed.


{'PIPELINE_pipeline_preprocessing_6b95373e-b472-4ebc-912b-d1ec90b3b8fc': [<taipy.core.job.job.Job at 0x257045668c0>],
 'PIPELINE_ARIMA_8a944c8c-a207-46eb-a52a-f696104281c1': [<taipy.core.job.job.Job at 0x2577220e920>,
 'PIPELINE_LinearRegression_a2695fc1-54b2-49f6-a43d-a8bf4081d910': [<taipy.core.job.job.Job at 0x2570732ec20>,
  <taipy.core.job.job.Job at 0x2570458ae00>]}

## Write in data nodes

To write a data node:

`<Data Node>.write(new_value)`

In [44]:
scenario.country.write('US')
tp.submit(scenario)
scenario.result.read()

C:\Users\jacta\AppData\Local\R-MINI~1\envs\notebook\lib\site-packages\taipy\core\data\csv.py:161: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(self._path)


[2022-10-05 09:41:25,086][Taipy][INFO] job JOB_task_preprocess_data_ce49b91a-0eea-4ca9-bf0d-23a2e501b244 is completed.


C:\Users\jacta\AppData\Local\Temp\ipykernel_41444\1815540409.py:16: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  .sum()\


[2022-10-05 09:41:25,260][Taipy][INFO] job JOB_task_train_02a67ea7-26d5-4cbc-9b8e-d7693b0d091d is completed.
[2022-10-05 09:41:25,346][Taipy][INFO] job JOB_task_forecast_3d4c9508-2b42-4b5f-857b-09ae3eab6615 is completed.
[2022-10-05 09:41:25,411][Taipy][INFO] job JOB_task_result_2ef65daa-ea0a-4aa0-a2ff-6ff3960eb413 is completed.
[2022-10-05 09:41:25,579][Taipy][INFO] job JOB_task_train_4abf9ec7-a8e4-449f-a549-bf67d2069daf is completed.
[2022-10-05 09:41:25,656][Taipy][INFO] job JOB_task_forecast_da69983a-53b3-44cc-8134-163f914f8f50 is completed.
[2022-10-05 09:41:25,711][Taipy][INFO] job JOB_task_result_e9bbd92c-e581-4bf9-b0bb-a8d76614e5b1 is completed.


,Date,Deaths,Months,Days,Week,Day of week,Predictions
0,2020-01-22,0.0,1,3,4,2,NaN
1,2020-01-23,0.0,1,4,4,3,NaN
2,2020-01-24,0.0,1,5,4,4,NaN
3,2020-01-25,0.0,1,6,4,5,NaN
4,2020-01-26,0.0,1,7,4,6,NaN
...,...,...,...,...,...,...,...
340,2020-12-27,334533.0,12,7,52,6,NaN
341,2020-12-28,336438.0,12,1,53,0,NaN
342,2020-12-29,340061.0,12,2,53,1,NaN
343,2020-12-30,343783.0,12,3,53,2,NaN


## Simple framework

In [45]:
scenario = tp.create_scenario(scenario_cfg, name='Second Scenario')
tp.submit(scenario)

C:\Users\jacta\AppData\Local\R-MINI~1\envs\notebook\lib\site-packages\taipy\core\data\csv.py:161: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(self._path)


[2022-10-05 09:41:27,197][Taipy][INFO] job JOB_task_preprocess_data_c05dd819-f7f6-41ad-a3be-13d4cccd7b34 is completed.


C:\Users\jacta\AppData\Local\Temp\ipykernel_41444\1815540409.py:16: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  .sum()\


[2022-10-05 09:41:27,363][Taipy][INFO] job JOB_task_train_ae5dffcd-2b12-428d-b343-4cd2c5253d89 is completed.
[2022-10-05 09:41:27,443][Taipy][INFO] job JOB_task_forecast_3a343ba4-9471-42cd-9b22-1135ae6916d1 is completed.
[2022-10-05 09:41:27,509][Taipy][INFO] job JOB_task_result_c1e8c620-4093-4eb6-9760-47e29872774c is completed.
[2022-10-05 09:41:27,668][Taipy][INFO] job JOB_task_train_8333d65a-4d99-44bd-adbb-7c36c6a08d2d is completed.
[2022-10-05 09:41:27,744][Taipy][INFO] job JOB_task_forecast_91756136-70ea-4471-bb3e-320d77176c2e is completed.
[2022-10-05 09:41:27,799][Taipy][INFO] job JOB_task_result_a340dad3-1488-42c8-aff8-92ffcd938b8a is completed.


{'PIPELINE_pipeline_preprocessing_eaf8ace3-5c18-44ea-aed8-f06535fb90c9': [<taipy.core.job.job.Job at 0x2570781a9b0>],
 'PIPELINE_ARIMA_824d24b9-4412-412d-8edb-88240b21e9cd': [<taipy.core.job.job.Job at 0x2570784fcd0>,
 'PIPELINE_LinearRegression_01246576-bf93-42f0-9f22-8cc4caceabf2': [<taipy.core.job.job.Job at 0x25711928d00>,
  <taipy.core.job.job.Job at 0x2571190fb50>]}

In [46]:
scenario.ARIMA.model.read()

LinearRegression()

In [47]:
scenario.pipelines['LinearRegression'].model.read()

LinearRegression()

In [48]:
[s.country.read() for s in tp.get_scenarios()]

['US', 'US', 'France', 'France', 'Finland']

In [49]:
[s.date.read() for s in tp.get_scenarios()]

[datetime.datetime(2020, 10, 10, 0, 0),
 datetime.datetime(2020, 10, 10, 0, 0),
 datetime.datetime(2020, 10, 10, 0, 0),
 datetime.datetime(2020, 10, 10, 0, 0),
 datetime.datetime(2020, 11, 1, 0, 0)]

## Create a Gui for the backend
_scenario_selector_ lets you choose a scenario and display its results.

In [50]:
scenario_selector = [(s.id, s.name) for s in tp.get_scenarios()]
selected_scenario = scenario.id
print(scenario_selector,'\n', selected_scenario)

[('SCENARIO_scenario_21c075d7-8e3c-42ca-af7a-f3318490d644', 'First Scenario'), ('SCENARIO_scenario_58a5cc7a-fdce-42ef-97aa-f1721841c331', 'First Scenario'), ('SCENARIO_scenario_70c7cd0a-de78-45d9-8eff-39c44c45653b', 'Second Scenario'), ('SCENARIO_scenario_f9cdfa16-293c-4e41-bfac-bcff70f70a3d', 'Second Scenario'), ('SCENARIO_scenario_fecbc342-ca11-4fae-b494-f7f1d230c9ce', 'test')] 
 SCENARIO_scenario_f9cdfa16-293c-4e41-bfac-bcff70f70a3d


In [51]:
result_arima = scenario.pipelines['ARIMA'].result.read()
result_rd = scenario.pipelines['LinearRegression'].result.read()
result = result_rd.merge(result_arima, on="Date", how="outer").sort_values(by='Date')
result

,Date,Deaths_x,Months_x,Days_x,Week_x,Day of week_x,Predictions_x,Deaths_y,Months_y,Days_y,Week_y,Day of week_y,Predictions_y
0,2020-01-24,0.0,1,5,4,4,NaN,0.0,1,5,4,4,NaN
1,2020-01-25,0.0,1,6,4,5,NaN,0.0,1,6,4,5,NaN
2,2020-01-26,0.0,1,7,4,6,NaN,0.0,1,7,4,6,NaN
3,2020-01-27,0.0,1,1,5,0,NaN,0.0,1,1,5,0,NaN
4,2020-01-28,0.0,1,2,5,1,NaN,0.0,1,2,5,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
338,2020-12-27,62867.0,12,7,52,6,NaN,62867.0,12,7,52,6,NaN
339,2020-12-28,63235.0,12,1,53,0,NaN,63235.0,12,1,53,0,NaN
340,2020-12-29,64204.0,12,2,53,1,NaN,64204.0,12,2,53,1,NaN
341,2020-12-30,64508.0,12,3,53,2,NaN,64508.0,12,3,53,2,NaN


**Tips** : the _value_by_id_ property if set to True for a selected will make _selected_scenario_ directly refer to the first element of the tupple (here the id)

![](img/predictions.png)

In [52]:
prediction_md = """
<|layout|columns=1 2 5 1 3|
<|{scenario_name}|input|label=Name|>

<br/>
<|Create|button|on_action=create_new_scenario|>

Prediction date
<|{first_date}|date|>

<|{selected_country}|selector|lov={country_lov}|dropdown|on_change=on_change_country|label=Country|>

<br/>
<|Submit|button|on_action=submit_scenario|>

<|{selected_scenario}|selector|lov={scenario_selector}|on_change=actualize_graph|dropdown|value_by_id|label=Scenario|>
|>

<|{result}|chart|x=Date|y[1]=Deaths_x|type[1]=bar|y[2]=Predictions_x|y[3]=Predictions_y|>
"""

In [53]:
def create_new_scenario(state):
    scenario = tp.create_scenario(scenario_cfg, name=state.scenario_name)
    state.scenario_selector += [(scenario.id, scenario.name)]

In [54]:
def submit_scenario(state):
    # 1) get the selected scenario
    # 2) write in country Data Node, the selected country
    # 3) submit the scenario
    # 4) actualize le graph avec actualize_graph
    scenario = tp.get(state.selected_scenario)
    scenario.country.write(state.selected_country)
    scenario.date.write(state.first_date.replace(tzinfo=None))
    tp.submit(scenario)
    actualize_graph(state)

In [55]:
def actualize_graph(state):
    # 1) update the result dataframe
    # 2) change selected_country with the predicted country of the scenario
    scenario = tp.get(state.selected_scenario)
    result_arima = scenario.pipelines['ARIMA'].result.read()
    result_rd = scenario.pipelines['LinearRegression'].result.read()
    if result_arima is not None and result_rd is not None:
        state.result = result_rd.merge(result_arima, on="Date", how="outer").sort_values(by='Date')
    state.selected_country = scenario.country.read()

In [56]:
Gui(prediction_md).run(dark_mode=False)

Gui server has been stopped.
 * Server starting on http://127.0.0.1:5000


# Multi-pages and Taipy Rest

To create a multi-pages app, we only need a dictionary with names as the keys and the Markdowns as the values.

The _navbar_ control (<|navbar|>) has a default behaviour. It redirects to the different pages of the app automatically. Other solutions exists.

![](img/multi_pages.png)

In [57]:
navbar_md = "<center>\n<|navbar|>\n</center>"

pages = {
    "Map":navbar_md+map_md,
    "Country":navbar_md+final_country_md,
    "Predictions":navbar_md+prediction_md
}

rest = tp.Rest()

gui_multi_pages = Gui(pages=pages)
tp.run(gui_multi_pages, rest, dark_mode=False, port=5006)

 * Server starting on http://127.0.0.1:5006
